# Testing Trained Models

In [7]:
!pwd

/home/amogh/cmu/notebooks


In [8]:
!ls scripts

affectnet-training.py  face_scripts.pyc  save_features.sh
affectnet-training.sh  __init__.py	 slurm-3561901.out
face_scripts.py        __init__.pyc	 slurm-3562661.out


In [9]:
import scripts.face_scripts as fs

loading libraries
libraries loaded


In [10]:
print("loading libraries")
import os, sys, random, glob, argparse, math, gc

import cv2
import dlib
import imutils
from imutils import face_utils
import matplotlib
# import matplotlib.pyplot as plt #causes segmentation fault, so do not use.
from skimage.feature import hog
from skimage import exposure

import sklearn
from sklearn import svm, metrics
import pickle

import numpy as np
import pandas as pd
from bcolz import carray

from tqdm import tqdm
from time import sleep
import datetime as dt
print("libraries loaded")

loading libraries
libraries loaded


In [11]:
folder_model = '/media/amogh/Stuff/CMU/datasets/DISFA_data/models/hpc_12July/models/'

In [12]:
list_model_path = glob.glob (folder_model+'/*/result.sav')
list_model_path

['/media/amogh/Stuff/CMU/datasets/DISFA_data/models/hpc_12July/models/2_FAU1_1/result.sav',
 '/media/amogh/Stuff/CMU/datasets/DISFA_data/models/hpc_12July/models/2_FAU2_1/result.sav',
 '/media/amogh/Stuff/CMU/datasets/DISFA_data/models/hpc_12July/models/2_FAU4_1/result.sav',
 '/media/amogh/Stuff/CMU/datasets/DISFA_data/models/hpc_12July/models/3_FAU1_1/result.sav',
 '/media/amogh/Stuff/CMU/datasets/DISFA_data/models/hpc_12July/models/3_FAU2_1/result.sav',
 '/media/amogh/Stuff/CMU/datasets/DISFA_data/models/hpc_12July/models/3_FAU4_1/result.sav']

### Loading Models

In [13]:
list_model = [[file_model.split('/')[-2], pickle.load(open(file_model,'rb'))] for file_model in list_model_path]

In [14]:
for path_file,model in list_model:
    print ("in model: thresh",path_file.split('/'))
    best_estimator = model.best_estimator_
    best_params = model.best_params_
    best_score = model.best_score_
#     cv_res = model.cv_results_
    print ("best estimator is: ",best_estimator)
    print ("best params are: ",best_params)    
    print ("best score is: ",best_score)
#     print ("cross validation results are: ", cv_res)
    print("")

('in model: thresh', ['2_FAU1_1'])
('best estimator is: ', SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))
('best params are: ', {'kernel': 'linear', 'C': 1.0, 'gamma': 0.001})
('best score is: ', 0.72746553552492044)

('in model: thresh', ['2_FAU2_1'])
('best estimator is: ', SVC(C=10.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))
('best params are: ', {'kernel': 'linear', 'C': 10.0, 'gamma': 0.001})
('best score is: ', 0.78091637010676151)

('in model: thresh', ['2_FAU4_1'])
('best estimator is: ', SVC(C=10.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='linear',
  max_iter

In [15]:
dict_model = dict(list_model)

In [16]:
dict_model

{'2_FAU1_1': GridSearchCV(cv=[(array([4339, 4340, ..., 9428, 9429]), array([   0,    1, ..., 4337, 4338])), (array([   0,    1, ..., 9428, 9429]), array([4339, 4340, ..., 4407, 4408])), (array([   0,    1, ..., 9428, 9429]), array([4409, 4410, ..., 4709, 4710])), (array([   0,    1, ..., 9428, 9429]), array([4711, 4712, ..., 7062, 7063])), (array([   0,    1, ..., 9428, 9429]), array([7064, 7065, ..., 8342, 8343])), (array([   0,    1, ..., 8342, 8343]), array([8344, 8345, ..., 9428, 9429]))],
        error_score='raise',
        estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
   decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
   max_iter=-1, probability=False, random_state=None, shrinking=True,
   tol=0.001, verbose=False),
        fit_params=None, iid=True, n_jobs=28,
        param_grid={'kernel': ['linear'], 'C': array([  0.1,   0.5,   1. ,   5. ,  10. ,  50. ]), 'gamma': array([  1.00000e-03,   5.00000e-03,   1.00000e-02,   5.00000e-02,
     

## Observations:
* Makes sense that the threshold 3 ones are easily separable with a higher accuracy

# Making predictions on kids data

In [17]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [18]:
def get_features(im_path,thresh, cropping_function_name, o=6 ,ppc=(8,8) ,cpb=(4,4), detector=detector, predictor=predictor,  function_dict=fs.function_dict, featuresFunction=fs.getHOGFeatures):
    try:
        #cropping and aligning images
        im_aligned_cropped,landmarkPoints = fs.detectAndaligncrop(im_path, detector, predictor)
        cropped_rgb_image = function_dict[cropping_function_name] (im_aligned_cropped, landmarkPoints)
        #getting features
        fd = featuresFunction(o, ppc, cpb, cropped_rgb_image)
        return fd
    except: 
#         print("error in file, possibly face not detected in : ", im_path)
        return None
    

In [19]:
def pred(im_path, thresh, cropping_function_name,dict_model=dict_model , o=6 ,ppc=(8,8) ,cpb=(4,4), detector=detector, predictor=predictor,  function_dict=fs.function_dict, featuresFunction=fs.getHOGFeatures):
    model = dict_model['{}_{}'.format(thresh, cropping_function_name)]
    fd = get_features(im_path=im_path,thresh=thresh, cropping_function_name=cropping_function_name, o=o ,ppc=ppc ,cpb=cpb, detector=detector, predictor=predictor,  function_dict=function_dict, featuresFunction=featuresFunction)
#     print(fd)
    if fd is not None:
        pred = model.predict([fd])
#         print("prediction is: " , pred)
        return pred
    else:
        return 2

Sample test

In [20]:
pred ('/media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_fau12/VIDEO_00-male-surprise_20180517_044827_472499497 092.jpg',3,'FAU1_1')

/home/amogh/anaconda3/envs/detect_face/lib/python2.7/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


array([1])

## For FAU 1 and 2

In [21]:
folder_fau_1_2_manual_labels = '/media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_fau12'

In [22]:
list_fau_1_2_manual_labels = glob.glob(folder_fau_1_2_manual_labels + "/*")

In [23]:
list_fau_1_2_manual_labels

['/media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_fau12/VIDEO_00-male-surprise_20180517_044827_472499497 085.jpg',
 '/media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_fau12/VIDEO_00-male-surprise_20180517_044827_472499497 086.jpg',
 '/media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_fau12/VIDEO_00-male-surprise_20180517_044827_472499497 087.jpg',
 '/media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_fau12/VIDEO_00-male-surprise_20180517_044827_472499497 088.jpg',
 '/media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_fau12/VIDEO_00-male-surprise_20180517_044827_472499497 089.jpg',
 '/media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_fau12/VIDEO_00-male-surprise_20180517_044827_472499497 090.jpg',
 '/media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_fau12/VIDEO_00-male-surprise_20180517_044827_472499497 091.jpg',
 '/media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_fau12/VIDEO_00-male-surprise_20180517_044827_472499497 092.jpg',
 '/media/amogh/Stuff/CMU

In [24]:
list_model_names = dict_model.keys()
list_model_names

['3_FAU2_1', '3_FAU4_1', '2_FAU2_1', '2_FAU1_1', '3_FAU1_1', '2_FAU4_1']

In [25]:
list_target = list_fau_1_2_manual_labels
for model in list_model_names:
    print("calculating")
    mod = dict_model[model]
    thresh =  model[0]
    crop_fun_name = model[2:]
    print ("thresh: ", thresh, crop_fun_name)
    array_results = ([pred(f,thresh,crop_fun_name) for f in list_target])
    print(array_results)
    print("positives are:{} out of {} ".format(array_results.count(1),len(array_results)), " and face not detected for {} ".format(array_results.count(2)))
    print("accuracy for images in which face is detected is: ", (array_results.count(1))/float(array_results.count(1)+array_results.count(0)) )
    

calculating
('thresh: ', '3', 'FAU2_1')
[2, 2, 2, 2, 2, array([1]), array([1]), array([1]), 2, 2, 2, array([1]), array([1]), array([1]), 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([1]), 2, array([0]), array([0]), array([0]), array([1]), array([0]), array([1]), array([0]), array([0]), array([0]), array([1]), array([1]), array([1]), array([1]), array([0]), array([1]), array([0]), array([1]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0])]
('positives are:15 out of 62 ', ' and face not detected for 19 ')
('accuracy for images in which face is detected is: ', 0.3488372093023256)
calculating
('thresh: ', '3', 'FAU4_1')
[2, 2, 2, 2, 2, array([1]), array([1]), array([1]), 2, 2, 2, array([1]), array([1]), array([1]), 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, array([1]), array([1]), array([1]), array([1]), array([1]), array([1]), array(

In [26]:
[1,1,2,4,3].count(1)

2

In [27]:
list_fau_1_2_manual_labels

['/media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_fau12/VIDEO_00-male-surprise_20180517_044827_472499497 085.jpg',
 '/media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_fau12/VIDEO_00-male-surprise_20180517_044827_472499497 086.jpg',
 '/media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_fau12/VIDEO_00-male-surprise_20180517_044827_472499497 087.jpg',
 '/media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_fau12/VIDEO_00-male-surprise_20180517_044827_472499497 088.jpg',
 '/media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_fau12/VIDEO_00-male-surprise_20180517_044827_472499497 089.jpg',
 '/media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_fau12/VIDEO_00-male-surprise_20180517_044827_472499497 090.jpg',
 '/media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_fau12/VIDEO_00-male-surprise_20180517_044827_472499497 091.jpg',
 '/media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_fau12/VIDEO_00-male-surprise_20180517_044827_472499497 092.jpg',
 '/media/amogh/Stuff/CMU